In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict

In [3]:
# Define the convolutional part of the architecture
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(
            in_channels=32, out_channels=64, kernel_size=3, padding=1
        )
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(
            in_channels=64, out_channels=128, kernel_size=3, padding=1
        )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        return x


# Define the recurrent part of the architecture
class RecurrentNet(nn.Module):
    def __init__(self):
        super(RecurrentNet, self).__init__()
        self.lstm = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        self.lstm.flatten_parameters()
        x, _ = self.lstm(x)
        x = torch.sigmoid(self.fc(x))
        return x


# Combine convolutional and recurrent parts into one model
class EndToEndModel(nn.Module):
    def __init__(self):
        super(EndToEndModel, self).__init__()
        self.conv_net = ConvNet()
        self.recurrent_net = RecurrentNet()

    def forward(self, x):
        x = self.conv_net(x)
        x = x.permute(
            0, 2, 1
        )  # Prepare for LSTM: (batch_size, sequence_length, features)
        x = self.recurrent_net(x)
        return x

In [4]:
from torchinfo import summary

model = EndToEndModel()

# Assume the input data shape is (batch_size, 6, 128)
input_shape = (32, 6, 128)  # batch_size, in_channels, sequence_length
summary(model, input_size=input_shape)

Layer (type:depth-idx)                   Output Shape              Param #
EndToEndModel                            [32, 32, 1]               --
├─ConvNet: 1-1                           [32, 128, 32]             --
│    └─Conv1d: 2-1                       [32, 32, 128]             992
│    └─MaxPool1d: 2-2                    [32, 32, 64]              --
│    └─Conv1d: 2-3                       [32, 64, 64]              6,208
│    └─MaxPool1d: 2-4                    [32, 64, 32]              --
│    └─Conv1d: 2-5                       [32, 128, 32]             24,704
├─RecurrentNet: 1-2                      [32, 32, 1]               --
│    └─LSTM: 2-6                         [32, 32, 128]             132,096
│    └─Linear: 2-7                       [32, 32, 1]               129
Total params: 164,129
Trainable params: 164,129
Non-trainable params: 0
Total mult-adds (M): 177.34
Input size (MB): 0.10
Forward/backward pass size (MB): 4.20
Params size (MB): 0.66
Estimated Total Size (MB): 4

In [5]:
# Pre-processing

In [6]:
# Augmentation

In [7]:
# LOSO

In [8]:
# Load .pkl data
def load_pkl_data(X_path, Y_path):
    with open(X_path, "rb") as f:
        X = pickle.load(f)  # List of numpy arrays
    with open(Y_path, "rb") as f:
        Y = pickle.load(f)  # List of numpy arrays
    return X, Y


# Dataset class
class IMUDataset(Dataset):
    def __init__(self, X, Y, sequence_length=128, downsample_factor=4):
        self.data = []
        self.labels = []
        self.sequence_length = sequence_length
        self.downsample_factor = downsample_factor
        self.subject_indices = []  # Record which subject each sample belongs to

        # Processing data for each session
        for subject_idx, (imu_data, labels) in enumerate(zip(X, Y)):
            imu_data = self.normalize(imu_data)
            num_samples = len(labels)

            for i in range(0, num_samples, sequence_length):
                imu_segment = imu_data[i : i + sequence_length]
                label_segment = labels[i : i + sequence_length]

                if len(imu_segment) == sequence_length:
                    self.data.append(imu_segment)
                    downsampled_labels = label_segment[:: self.downsample_factor]
                    self.labels.append(downsampled_labels)
                    self.subject_indices.append(subject_idx)

    def normalize(self, data):
        mean = np.mean(data, axis=0)
        std = np.std(data, axis=0)
        return (data - mean) / (std + 1e-5)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return x, y


# Paths to .pkl files
X_path = "./dataverse_files/pkl_data/pkl_data/DX_I_X.pkl"
Y_path = "./dataverse_files/pkl_data/pkl_data/DX_I_Y.pkl"

# Load data
X, Y = load_pkl_data(X_path, Y_path)

# Prepare dataset and dataloader
dataset = IMUDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
# Check the format and size of the data
print(f"Type of X: {type(X)}")
print(f"Type of Y: {type(Y)}")
print(f"Number of sessions in X: {len(X)}")
print(f"Number of sessions in Y: {len(Y)}")

# Check the shape of the first session
if len(X) > 0 and len(Y) > 0:
    print(f"Shape of the first session in X: {X[0].shape}")
    print(f"Shape of the first session in Y: {Y[0].shape}")

# Example: Iterate through the dataloader
for batch_x, batch_y in dataloader:
    print("Batch X shape:", batch_x.shape)
    print("Batch Y shape:", batch_y.shape)
    break

Type of X: <class 'list'>
Type of Y: <class 'list'>
Number of sessions in X: 15
Number of sessions in Y: 15
Shape of the first session in X: (827243, 6)
Shape of the first session in Y: (827243,)
Batch X shape: torch.Size([32, 128, 6])
Batch Y shape: torch.Size([32, 32])


In [ ]:
def train_model(model, train_loader, criterion, optimizer, device, epoch, num_epochs):
    model.train()
    running_loss = 0.0
    all_predictions = []
    all_labels = []

    for batch_x, batch_y in train_loader:
        batch_x = batch_x.permute(0, 2, 1).to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_x)
        outputs = outputs.squeeze(-1)

        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Collect predictions and labels for calculating metrics
        predictions = (outputs > 0.5).float().cpu().numpy()
        all_predictions.extend(predictions.flatten())
        all_labels.extend(batch_y.cpu().numpy().flatten())

    # Calculating training metrics
    metrics = calculate_metrics(all_labels, all_predictions)

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(
        f"Training - Loss: {avg_loss:.4f}, Accuracy: {metrics['accuracy']:.4f}, "
        f"Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, "
        f"F1: {metrics['f1']:.4f}"
    )

    return metrics


def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.permute(0, 2, 1).to(device)
            batch_y = batch_y.to(device)

            outputs = model(batch_x)
            outputs = outputs.squeeze(-1)

            loss = criterion(outputs, batch_y)
            running_loss += loss.item()

            predictions = (outputs > 0.5).float().cpu().numpy()
            all_predictions.extend(predictions.flatten())
            all_labels.extend(batch_y.cpu().numpy().flatten())

    metrics = calculate_metrics(all_labels, all_predictions)
    avg_loss = running_loss / len(test_loader)

    print(
        f"Testing - Loss: {avg_loss:.4f}, Accuracy: {metrics['accuracy']:.4f}, "
        f"Precision: {metrics['precision']:.4f}, Recall: {metrics['recall']:.4f}, "
        f"F1: {metrics['f1']:.4f}"
    )

    return metrics


def calculate_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }


# LOSO cross validation main function
def loso_cross_validation(X, Y, config):
    num_subjects = len(X)
    all_metrics = defaultdict(list)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for test_subject in range(num_subjects):
        print(f"\nTesting on subject {test_subject + 1}/{num_subjects}")

        # Preparing training and testing data
        train_X = [x for i, x in enumerate(X) if i != test_subject]
        train_Y = [y for i, y in enumerate(Y) if i != test_subject]
        test_X = [X[test_subject]]
        test_Y = [Y[test_subject]]

        # Creating a dataset and data loader
        train_dataset = IMUDataset(
            train_X,
            train_Y,
            sequence_length=config["sequence_length"],
            downsample_factor=config["downsample_factor"],
        )
        test_dataset = IMUDataset(
            test_X,
            test_Y,
            sequence_length=config["sequence_length"],
            downsample_factor=config["downsample_factor"],
        )

        train_loader = DataLoader(
            train_dataset, batch_size=config["batch_size"], shuffle=True
        )
        test_loader = DataLoader(
            test_dataset, batch_size=config["batch_size"], shuffle=False
        )

        # Initialize the model and optimizer
        model = EndToEndModel().to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.RMSprop(model.parameters(), lr=config["learning_rate"])

        # Training the model
        best_f1 = 0
        best_metrics = None

        for epoch in range(config["num_epochs"]):
            train_metrics = train_model(
                model,
                train_loader,
                criterion,
                optimizer,
                device,
                epoch,
                config["num_epochs"],
            )
            test_metrics = evaluate_model(model, test_loader, criterion, device)

            # Save the best model
            if test_metrics["f1"] > best_f1:
                best_f1 = test_metrics["f1"]
                best_metrics = test_metrics
                torch.save(model.state_dict(), f"best_model_subject_{test_subject}.pth")

        # Record the best results
        for metric, value in best_metrics.items():
            all_metrics[metric].append(value)

        print(f"\nBest metrics for subject {test_subject}:")
        for metric, value in best_metrics.items():
            print(f"{metric}: {value:.4f}")

    # Print overall results
    print("\nOverall LOSO Cross-validation Results:")
    for metric, values in all_metrics.items():
        mean_value = np.mean(values)
        std_value = np.std(values)
        print(f"{metric}: {mean_value:.4f} ± {std_value:.4f}")

    return all_metrics

In [11]:
# Loading data
X_path = "./dataverse_files/pkl_data/pkl_data/DX_I_X.pkl"
Y_path = "./dataverse_files/pkl_data/pkl_data/DX_I_Y.pkl"
X, Y = load_pkl_data(X_path, Y_path)

# Configuration parameters
config = {
    "sequence_length": 128,
    "downsample_factor": 4,
    "batch_size": 128,
    "learning_rate": 1e-3,
    "num_epochs": 20,
}

# Run LOSO cross validation
metrics = loso_cross_validation(X, Y, config)


Testing on subject 1/15
Epoch [1/20]
Training - Loss: 0.0985, Accuracy: 0.9616, Precision: 0.6057, Recall: 0.4206, F1: 0.4965
Testing - Loss: 0.0331, Accuracy: 0.9922, Precision: 0.1859, Recall: 0.0615, F1: 0.0924
Epoch [2/20]
Training - Loss: 0.0745, Accuracy: 0.9743, Precision: 0.7703, Recall: 0.6117, F1: 0.6819
Testing - Loss: 0.0887, Accuracy: 0.9735, Precision: 0.1527, Recall: 0.6844, F1: 0.2497
Epoch [3/20]
Training - Loss: 0.0658, Accuracy: 0.9766, Precision: 0.7927, Recall: 0.6483, F1: 0.7132
Testing - Loss: 0.0298, Accuracy: 0.9933, Precision: 0.4385, Recall: 0.1282, F1: 0.1984
Epoch [4/20]
Training - Loss: 0.0618, Accuracy: 0.9782, Precision: 0.8017, Recall: 0.6838, F1: 0.7381
Testing - Loss: 0.0926, Accuracy: 0.9680, Precision: 0.1134, Recall: 0.5810, F1: 0.1898
Epoch [5/20]
Training - Loss: 0.0603, Accuracy: 0.9793, Precision: 0.8063, Recall: 0.7106, F1: 0.7554
Testing - Loss: 0.0267, Accuracy: 0.9944, Precision: 0.9120, Recall: 0.1477, F1: 0.2542
Epoch [6/20]
Training - L